# Noise model selection on NANOGrav pulsars

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import numpy as np
import glob, os, json, string, pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
import logging, inspect, copy
logging.basicConfig(level=logging.WARNING)

In [3]:
import enterprise
from enterprise.pulsar import Pulsar

import enterprise_extensions
from enterprise_extensions import models, model_utils, blocks
from enterprise_extensions.models import model_singlepsr_noise
from enterprise_extensions.chromatic import solar_wind, chromatic
from enterprise_extensions.hypermodel import HyperModel

You'll need sksparse for get_coefficients() with common signals!


# Red-noise model selection on 12.5yr Dataset

## Get par, tim, and noise files

In [ ]:
# psr = Pulsar('./partim_no_noise/J0613-0200_NANOGrav_11yv0.gls.strip.par',
#              './partim_no_noise/J0613-0200_NANOGrav_11yv0.tim',
#               ephem='DE436')

# noisefiles = sorted(glob.glob('../11yr_stochastic_analysis/nano11y_data/noisefiles/*.json'))

# params = {}
# for noisefil in noisefiles:
#     with open(noisefil, 'r') as fp:
#         params.update(json.load(fp))

## Load Pickle File 

In [ ]:
psrname = 'B1855+09'#'J1911+1347'
filepath = './no_dmx_pickles/'
filepath += '{0}_ng12p5yr_v3_nodmx_ePSR.pkl'.format(psrname)
with open(filepath,'rb') as fin:
    psr=pickle.load(fin)

# Testing models with GP DM variations

__Very Import:__ What follows is an __example__ of noise model selection. For *most* pulsars the choice of noise models used in any given model selection analysis will be different than the ones chosen here. Those working on pulsars highlighted in the 11-year noise model analysis should include those models in their analyses and also use the best combination of models from that work in any final model selection that is done.

## Setup GP model selection

Choices for `psd`s (i.e. `diag` kernels):
	* `powerlaw`
	* `spectrum`
	* `tprocess`
	* `tprocess_adapt`
	
Choices for `nondiag` kernels:
	* `sq_exp`
	* `periodic`
	* `sq_exp_rfband`
	* `periodic_rfband`
	
Running a chromatic gp:
	* set `chrom_gp=True` or in the `kwarg` dictionary {`chrom_gp`:True}
	* set which general type of kernel you'll use `chrom_gp_kernel`= `['diag','nondiag']`
	* set the specific kernel, either `chrom_kernel='sq_exp'` or `chrom_psd='turnover'`, for instance
	* set the index, 1/f^idx, Default is 4, `chrom_idx=4.4`

```
dipcusp_kwargs = {'dm_expdip':True,
                  'dmexp_sign': 'negative',
                  'num_dmdips':2,
                  'dm_cusp_idx':[2,4],
                  'dm_expdip_tmin':[54700,57450],
                  'dm_expdip_tmax':[54850,57560],
                  'dmdip_seqname':'ism',
                  'dm_cusp':False,
                  'dm_cusp_sign':'negative',
                  'dm_cusp_idx':[2,4],
                  'dm_cusp_sym':False,
                  'dm_cusp_tmin':None,
                  'dm_cusp_tmax':None,
                  'num_dm_cusps':2, 
                  'dm_dual_cusp':True,
                  'dm_dual_cusp_tmin':[54700,57450],
                  'dm_dual_cusp_tmax':[54850,57560],}
```

In [6]:
red_psd = 'powerlaw'
dm_nondiag_kernel = ['periodic_rfband','sq_exp_rfband']#,'sq_exp', 'periodic']
dm_sw_gp = False
dm_annual = False
chrom_gp = True
chrom_gp_kernel = 'nondiag'
chrom_kernels= ['periodic','sq_exp']
white_vary = True

Use the inspect package to pull the arguments from `model_singlepsr_noise` and make a template for the keyword arguments (kwargs) dictionary we will be using to keep track of these various models. 

In [7]:
args = inspect.getfullargspec(model_singlepsr_noise)
keys = args[0][1:]
vals = args[3]
model_template = dict(zip(keys,vals))
model_template

{'tm_var': False,
 'tm_linear': False,
 'tmparam_list': None,
 'red_var': True,
 'psd': 'powerlaw',
 'red_select': None,
 'noisedict': None,
 'tm_svd': False,
 'tm_norm': True,
 'white_vary': True,
 'components': 30,
 'upper_limit': False,
 'wideband': False,
 'gamma_val': None,
 'dm_var': False,
 'dm_type': 'gp',
 'dmgp_kernel': 'diag',
 'dm_psd': 'powerlaw',
 'dm_nondiag_kernel': 'periodic',
 'dm_dt': 15,
 'dmx_data': None,
 'dm_annual': False,
 'gamma_dm_val': None,
 'chrom_gp': False,
 'chrom_gp_kernel': 'nondiag',
 'chrom_psd': 'powerlaw',
 'chrom_idx': 4,
 'chrom_kernel': 'periodic',
 'chrom_dt': 15,
 'dm_expdip': False,
 'dmexp_sign': 'negative',
 'dm_expdip_idx': 2,
 'dm_expdip_tmin': None,
 'dm_expdip_tmax': None,
 'num_dmdips': 1,
 'dmdip_seqname': None,
 'dm_cusp': False,
 'dm_cusp_sign': 'negative',
 'dm_cusp_idx': 2,
 'dm_cusp_sym': False,
 'dm_cusp_tmin': None,
 'dm_cusp_tmax': None,
 'num_dm_cusps': 1,
 'dm_cusp_seqname': None,
 'dm_dual_cusp': False,
 'dm_dual_cusp_tmin

Here we show one work flow where we set up a `for` loop to go through the various models. Make sure to save the `model_labels` and `model_kwargs`. The former will be useful for making noise flower plots, while the latter will be the final product for a pulsar in this analysis.

In [ ]:
# Create list of pta models for our model selection
nmodels = len(chrom_kernels) * len(dm_nondiag_kernel)
mod_index = np.arange(nmodels)

ptas = dict.fromkeys(mod_index)
model_dict = {}
model_labels = []
ct = 0
for dm in dm_nondiag_kernel:
    for chrom_kernel in chrom_kernels:
        if dm == 'None':
            dm_var = False
        else:
            dm_var = True
        # Copy template kwargs dict and replace values we are changing. 
        kwargs = copy.deepcopy(model_template)

        kwargs.update({'dm_var':dm_var,
                       'dmgp_kernel':'nondiag',
                       'psd':red_psd,
                       'white_vary':white_vary,
                       'dm_nondiag_kernel':dm,
                       'dm_sw_deter':True,
                       'dm_sw_gp':dm_sw_gp,
                       'dm_annual': dm_annual,
                       'swgp_basis': 'powerlaw',
                       'chrom_gp_kernel':chrom_gp_kernel,
                       'chrom_kernel':chrom_kernel,
                       'chrom_gp':chrom_gp})

        # Instantiate single pulsar noise model
        ptas[ct] = model_singlepsr_noise(psr, **kwargs)

        # Add labels and kwargs to save for posterity and plotting.
        model_labels.append([string.ascii_uppercase[ct],dm, chrom_kernel])
        model_dict.update({str(ct):kwargs})
        ct += 1

In [ ]:
# Instantiate a collection of models
super_model = HyperModel(ptas)

In [ ]:
super_model.params

In [ ]:
model_labels

## Set the out directory for you chains and other sampler setup
### !!! Important !!! Please set the chain directory outside of the git repository (easier) or at least do not try and commit your chains to the repo. 

In [ ]:
round_number = '5_powerlaw_chrom_gp_periodic_vs_sq_exp_and_periodic_rfband_vs_sq_exp_rfband'
outdir = './chains/{}/round_{}'.format(psr.name,round_number)
print('Will Save to: ',outdir)
emp_distr_path = './wn_emp_dists/{0}_ng12p5yr_v3_std_plaw_emp_dist.pkl'.format(psr.name)
sampler = super_model.setup_sampler(resume=True, outdir=outdir,
                                    empirical_distr=emp_distr_path)

In [ ]:
model_params = {}
for ky, pta in ptas.items():
    model_params.update({str(ky) : pta.param_names})

In [ ]:
with open(outdir+'/model_params.json' , 'w') as fout:
    json.dump(model_params, fout, sort_keys=True,
              indent=4, separators=(',', ': '))
    
with open(outdir+'/model_kwargs.json' , 'w') as fout:
    json.dump(model_dict, fout, sort_keys=True,
              indent=4, separators=(',', ': '))
    
with open(outdir+'/model_labels.json' , 'w') as fout:
    json.dump(model_labels, fout, sort_keys=True,
              indent=4, separators=(',', ': '))

In [ ]:
# sampler for N steps
N = int(5e6)
x0 = super_model.initial_sample()

In [ ]:
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, burn=100000)

## Example with chromatic models. 

The analysis above seems to indicate that `J1600-3053` wants an annual-DM term whenever a Fourier basis representation of the DM variations is used. We use that informatio in the following. We also fix the red-noise PSD to be a `power-law`.

In [ ]:
psrname = 'B1855+09'#'J1600-3053'
filepath = './no_dmx_pickles/'
filepath += '{0}_ng12p5yr_v3_nodmx_ePSR.pkl'.format(psrname)
with open(filepath,'rb') as fin:
    psr=pickle.load(fin)

In [ ]:
# Create list of pta models for our model selection
ptas = {}
model_labels = []
model_dict = {}

kwargs = copy.deepcopy(model_template)            
kwargs.update({'dm_var':True,
               'dmgp_kernel':'nondiag',
               'white_vary':True,
               'dm_nondiag_kernel':'sq_exp_rfband',
               'dm_sw_deter':True,
               })
            
# Instantiate single pulsar noise model
ptas[0] = model_singlepsr_noise(psr, **kwargs)

# Add labels and kwargs to save for posterity and plotting.
# Model, DM Kernel, Chromatic, Chrom Idx
model_labels.append(['A','sq_exp_rfband', False, None])
model_dict.update({str(0):kwargs})

kwargs = copy.deepcopy(model_template)            
kwargs.update({'dm_var':True,
               'dmgp_kernel':'nondiag',
               'white_vary':True,
               'dm_nondiag_kernel':'periodic_rfband',
               'dm_sw_deter':True,
               })
            
# Instantiate single pulsar noise model
ptas[1] = model_singlepsr_noise(psr, **kwargs)

# Add labels and kwargs to save for posterity and plotting.
# Model, DM Kernel, Chromatic, Chrom Idx
model_labels.append(['B','periodic_rfband', False, None])
model_dict.update({str(1):kwargs})

kwargs = copy.deepcopy(model_template)            
kwargs.update({'dm_var':True,
               'dmgp_kernel':'nondiag',
               'white_vary':True,
               'dm_nondiag_kernel':'sq_exp_rfband',
               'chrom_gp':True,
               'chrom_gp_kernel':'nondiag',
               'chrom_kernel':'sq_exp',
               'dm_sw_deter':True,
               })
# Instantiate single pulsar noise model
ptas[2] = model_singlepsr_noise(psr, **kwargs)

# Add labels and kwargs to save for posterity and plotting.
# Model, DM Kernel, Chromatic, Chrom Idx
model_labels.append(['C','sq_exp_rfband', True, 'sq_exp'])
model_dict.update({str(2):kwargs})


kwargs = copy.deepcopy(model_template)            
kwargs.update({'dm_var':True,
               'dmgp_kernel':'nondiag',
               'white_vary':True,
               'dm_nondiag_kernel':'sq_exp_rfband',
               'chrom_gp':True,
               'chrom_gp_kernel':'nondiag',
               'chrom_kernel':'sq_exp',
               'dm_sw_deter':True,
               })

# Instantiate single pulsar noise model
ptas[3] = model_singlepsr_noise(psr, **kwargs)

# Add labels and kwargs to save for posterity and plotting.
# Model, DM Kernel, Chromatic, Chrom Idx
model_labels.append(['D','sq_exp_rfband', True, 'periodic'])
model_dict.update({str(3):kwargs})

In [ ]:
# Instanciate a collection of models
super_model = HyperModel(ptas)

In [ ]:
outdir = './chains/{}/chrom_dmgp/'.format(psr.name)
emp_distr_path = './wn_emp_dists/{0}_ng12p5yr_v3_std_plaw_emp_dist.pkl'.format(psr.name)
sampler = super_model.setup_sampler(resume=True, outdir=outdir,
                                    empirical_distr=emp_distr_path)

In [ ]:
model_params = {}
for ky, pta in ptas.items():
    model_params.update({str(ky) : pta.param_names})

In [ ]:
with open(outdir+'/model_params.json' , 'w') as fout:
    json.dump(model_params, fout, sort_keys=True,
              indent=4, separators=(',', ': '))
    
with open(outdir+'/model_kwargs.json' , 'w') as fout:
    json.dump(model_dict, fout, sort_keys=True,
              indent=4, separators=(',', ': '))
    
with open(outdir+'/model_labels.json' , 'w') as fout:
    json.dump(model_labels, fout, sort_keys=True,
              indent=4, separators=(',', ': '))

In [ ]:
# sampler for N steps
N = int(5e6)
x0 = super_model.initial_sample()

In [ ]:
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, burn=200000)